In [8]:
"""
!pip install transformers[torch]
!pip install accelerate -U
"""

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 7.8 MB/s eta 0:00:00ta 0:00:01

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm
import shap
import joblib
import xgboost as xgb
"""
# Load data
data_discharge_notes = pd.read_csv("/kaggle/input/mimic-iv-prime/discharge.csv")
data_mimic_patients = pd.read_csv('/kaggle/input/mimic-iv-prime/patients.csv')
"""

data_mimic_patients = pd.read_csv('/home/iqh4001/Iqram_WCM/MADE Lab/LLM_EHR/Data/mimic-iv-2.2/hosp/patients.csv')

data_discharge_notes = pd.read_csv('/home/iqh4001/Iqram_WCM/MADE Lab/LLM_EHR/Data/MIMIC-IV Data/discharge.csv')

#data_discharge = pd.read_csv('/home/iqh4001/Iqram_WCM/MADE Lab/LLM_EHR/Data/MIMIC-IV Data/discharge_detail.csv')

# Merge data_discharge_notes with data_mimic_patients on 'subject_id'
data_discharge_notes_1 = data_discharge_notes.merge(data_mimic_patients[['subject_id', 'dod']], on='subject_id', how='left')


In [2]:
data_discharge_notes_1

,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text,dod
0,10000032-DS-21,10000032,22595853,DS,21,2180-05-07 00:00:00,2180-05-09 15:26:00,\nName: ___ Unit No: _...,2180-09-09
1,10000032-DS-22,10000032,22841357,DS,22,2180-06-27 00:00:00,2180-07-01 10:15:00,\nName: ___ Unit No: _...,2180-09-09
2,10000032-DS-23,10000032,29079034,DS,23,2180-07-25 00:00:00,2180-07-25 21:42:00,\nName: ___ Unit No: _...,2180-09-09
3,10000032-DS-24,10000032,25742920,DS,24,2180-08-07 00:00:00,2180-08-10 05:43:00,\nName: ___ Unit No: _...,2180-09-09
4,10000084-DS-17,10000084,23052089,DS,17,2160-11-25 00:00:00,2160-11-25 15:09:00,\nName: ___ Unit No: __...,2161-02-13
...,...,...,...,...,...,...,...,...,...
331788,19999828-DS-6,19999828,29734428,DS,6,2147-08-04 00:00:00,2147-08-12 15:36:00,\nName: ___ Unit No: ___...,NaN
331789,19999828-DS-7,19999828,25744818,DS,7,2149-01-18 00:00:00,2149-01-19 07:03:00,\nName: ___ Unit No: ___...,NaN
331790,19999840-DS-20,19999840,26071774,DS,20,2164-07-28 00:00:00,2164-07-29 14:52:00,\nName: ___ Unit No: ___\...,2164-09-17
331791,19999840-DS-21,19999840,21033226,DS,21,2164-09-17 00:00:00,2164-09-18 01:36:00,\nName: ___ Unit No: ___\...,2164-09-17


In [3]:
# Convert 'charttime' and 'dod' to datetime format
data_discharge_notes_1['charttime'] = pd.to_datetime(data_discharge_notes_1['charttime'])
data_discharge_notes_1['dod'] = pd.to_datetime(data_discharge_notes_1['dod'])

# Calculate the difference in days between 'dod' and 'charttime'
data_discharge_notes_1['Out_of_hospital_mortality_days'] = (data_discharge_notes_1['dod'] - data_discharge_notes_1['charttime']).dt.days



In [4]:
data_discharge_notes_1

,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text,dod,Out_of_hospital_mortality_days
0,10000032-DS-21,10000032,22595853,DS,21,2180-05-07,2180-05-09 15:26:00,\nName: ___ Unit No: _...,2180-09-09,125.0
1,10000032-DS-22,10000032,22841357,DS,22,2180-06-27,2180-07-01 10:15:00,\nName: ___ Unit No: _...,2180-09-09,74.0
2,10000032-DS-23,10000032,29079034,DS,23,2180-07-25,2180-07-25 21:42:00,\nName: ___ Unit No: _...,2180-09-09,46.0
3,10000032-DS-24,10000032,25742920,DS,24,2180-08-07,2180-08-10 05:43:00,\nName: ___ Unit No: _...,2180-09-09,33.0
4,10000084-DS-17,10000084,23052089,DS,17,2160-11-25,2160-11-25 15:09:00,\nName: ___ Unit No: __...,2161-02-13,80.0
...,...,...,...,...,...,...,...,...,...,...
331788,19999828-DS-6,19999828,29734428,DS,6,2147-08-04,2147-08-12 15:36:00,\nName: ___ Unit No: ___...,NaT,NaN
331789,19999828-DS-7,19999828,25744818,DS,7,2149-01-18,2149-01-19 07:03:00,\nName: ___ Unit No: ___...,NaT,NaN
331790,19999840-DS-20,19999840,26071774,DS,20,2164-07-28,2164-07-29 14:52:00,\nName: ___ Unit No: ___\...,2164-09-17,51.0
331791,19999840-DS-21,19999840,21033226,DS,21,2164-09-17,2164-09-18 01:36:00,\nName: ___ Unit No: ___\...,2164-09-17,0.0


In [5]:
# Prepare the text data (X) and the target variable (y)
X = data_discharge_notes_1['text']
y = data_discharge_notes_1['Out_of_hospital_mortality_days']


In [6]:
valid_indices = y.dropna().index
X = X.loc[valid_indices].reset_index(drop=True)
y = y.dropna().reset_index(drop=True)

# Basic BERT

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from tqdm import tqdm

# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("Selected Device: CUDA")
else:
    device = torch.device('cpu')
    print("Selected Device: CPU")

# Load pre-trained BERT tokenizer
print("_-_-_-_-_-Tokenizing_-_-_-_-_-_-")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Example medical text data (X) and continuous target values (y)
texts = X.to_list()
values = y.to_list()

# Tokenize text data with a progress bar
print("_-_-_-_-_-Encodings-_-_-_-_-_-")
encodings = {"input_ids": [], "attention_mask": []}

for text in tqdm(texts, desc="Tokenizing"):
    encoding = tokenizer(text, truncation=True, padding='max_length', max_length=512)
    encodings["input_ids"].append(encoding["input_ids"])
    encodings["attention_mask"].append(encoding["attention_mask"])

# Convert lists to tensors
encodings = {key: torch.tensor(val) for key, val in encodings.items()}

# Create a dataset class
class MedicalDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, values):
        self.encodings = encodings
        self.values = values

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.values[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.values)

print("_-_-_-_-_-MedicalDataset Creation-_-_-_-_-_-")
dataset = MedicalDataset(encodings, values)

# Split into train and test sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
print("_-_-_-_-_-Splitting into train and test sets-_-_-_-_-_-")
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Load pre-trained BERT model with a regression head and move to GPU
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1).to(device)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',          
    num_train_epochs=3,              
    per_device_train_batch_size=8,   
    per_device_eval_batch_size=8,    
    warmup_steps=500,                
    weight_decay=0.01,               
    logging_dir='./logs',            
    logging_strategy='epoch',  # Use 'epoch' for logging strategy
    save_strategy='epoch',     # Ensure save and eval strategies match
    evaluation_strategy='epoch', # Set evaluation strategy to 'epoch'
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss"
)

# Define custom loss function for regression
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.squeeze()
    mse = ((preds - labels) ** 2).mean().item()
    return {'mse': mse}

# Initialize Trainer
trainer = Trainer(
    model=model,                        
    args=training_args,                 
    train_dataset=train_dataset,         
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Train the model
print("_-_-_-_-_-Training Started_-_-_-_-_-_-")
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print(results)


Selected Device: CUDA
_-_-_-_-_-Tokenizing_-_-_-_-_-_-
_-_-_-_-_-Encodings-_-_-_-_-_-


Tokenizing:   3%|▎         | 3348/95707 [01:39<45:33, 33.79it/s]  


KeyboardInterrupt: 

# BioBERT

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from tqdm import tqdm  # Import tqdm for progress bar

# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("Selected Device: CUDA")
else:
    device = torch.device('cpu')
    print("Selected Device: CPU")

# Load pre-trained BioBERT tokenizer
print("_-_-_-_-_-Tokenizing_-_-_-_-_-_-")
tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-base-cased-v1.1')

# Example medical text data (X) and continuous target values (y)
texts = X.to_list()
values = y.to_list()

# Tokenize text data with a progress bar
print("_-_-_-_-_-Encodings-_-_-_-_-_-")
encodings = {'input_ids': [], 'attention_mask': []}

# Use tqdm to show progress
for text in tqdm(texts, desc="Tokenizing"):
    encoding = tokenizer(text, truncation=True, padding='max_length', max_length=512)
    encodings['input_ids'].append(encoding['input_ids'])
    encodings['attention_mask'].append(encoding['attention_mask'])

# Convert lists to tensors
encodings = {key: torch.tensor(val) for key, val in encodings.items()}

# Create a dataset class
class MedicalDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, values):
        self.encodings = encodings
        self.values = values

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.values[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.values)

print("_-_-_-_-_-MedicalDataset Creation-_-_-_-_-_-")
dataset = MedicalDataset(encodings, values)

# Split into train and test sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
print("_-_-_-_-_-Splitting into train and test sets-_-_-_-_-_-")
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Load pre-trained BioBERT model with a regression head and move to GPU
model = BertForSequenceClassification.from_pretrained('dmis-lab/biobert-base-cased-v1.1', num_labels=1).to(device)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',          
    num_train_epochs=3,              
    per_device_train_batch_size=8,   
    per_device_eval_batch_size=8,    
    warmup_steps=500,                
    weight_decay=0.01,               
    logging_dir='./logs',            
    logging_strategy='epoch',
    eval_strategy='epoch',  # Updated parameter name
    logging_steps=10,
    save_strategy='epoch',  # Ensure save_strategy matches eval_strategy
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss"
)

# Define custom loss function for regression
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.squeeze()
    mse = ((preds - labels) ** 2).mean().item()
    return {'mse': mse}

# Initialize Trainer
trainer = Trainer(
    model=model,                        
    args=training_args,                 
    train_dataset=train_dataset,         
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Train the model
print("_-_-_-_-_-Training Started_-_-_-_-_-_-")
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print(results)


Selected Device: CUDA
_-_-_-_-_-Tokenizing_-_-_-_-_-_-
_-_-_-_-_-Encodings-_-_-_-_-_-


Tokenizing: 100%|██████████| 100/100 [00:07<00:00, 12.69it/s]


_-_-_-_-_-MedicalDataset Creation-_-_-_-_-_-
_-_-_-_-_-Splitting into train and test sets-_-_-_-_-_-


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


_-_-_-_-_-Training Started_-_-_-_-_-_-


Epoch,Training Loss,Validation Loss,Mse
1,903530.000000,1514880.875000,1514880.875000
2,903416.100000,1514652.375000,1514652.625000
3,903089.300000,1514253.250000,1514253.250000


{'eval_loss': 1514253.25, 'eval_mse': 1514253.25, 'eval_runtime': 0.299, 'eval_samples_per_second': 66.881, 'eval_steps_per_second': 10.032, 'epoch': 3.0}


# PubMedBERT

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from tqdm import tqdm  # Import tqdm for progress bar

# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("Selected Device: CUDA")
else:
    device = torch.device('cpu')
    print("Selected Device: CPU")

# Load pre-trained PubMedBERT tokenizer
print("_-_-_-_-_-Tokenizing_-_-_-_-_-_-")
tokenizer = BertTokenizer.from_pretrained('microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext')

# Example medical text data (X) and continuous target values (y)
texts = X.to_list()
values = y.to_list()

# Tokenize text data with a progress bar
print("_-_-_-_-_-Encodings-_-_-_-_-_-")
encodings = {'input_ids': [], 'attention_mask': []}

# Use tqdm to show progress
for text in tqdm(texts, desc="Tokenizing"):
    encoding = tokenizer(text, truncation=True, padding='max_length', max_length=512)
    encodings['input_ids'].append(encoding['input_ids'])
    encodings['attention_mask'].append(encoding['attention_mask'])

# Convert lists to tensors
encodings = {key: torch.tensor(val) for key, val in encodings.items()}

# Create a dataset class
class MedicalDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, values):
        self.encodings = encodings
        self.values = values

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.values[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.values)

print("_-_-_-_-_-MedicalDataset Creation-_-_-_-_-_-")
dataset = MedicalDataset(encodings, values)

# Split into train and test sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
print("_-_-_-_-_-Splitting into train and test sets-_-_-_-_-_-")
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Load pre-trained PubMedBERT model with a regression head and move to GPU
model = BertForSequenceClassification.from_pretrained('microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext', num_labels=1).to(device)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',          
    num_train_epochs=3,              
    per_device_train_batch_size=8,   
    per_device_eval_batch_size=8,    
    warmup_steps=500,                
    weight_decay=0.01,               
    logging_dir='./logs',            
    logging_strategy='epoch',        # Update deprecated argument
    evaluation_strategy='epoch',     # Update deprecated argument
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    save_strategy='epoch'            # Ensure save strategy matches eval strategy
)

# Define custom loss function for regression
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.squeeze()
    mse = ((preds - labels) ** 2).mean().item()
    return {'mse': mse}

# Initialize Trainer
trainer = Trainer(
    model=model,                        
    args=training_args,                 
    train_dataset=train_dataset,         
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Train the model
print("_-_-_-_-_-Training Started_-_-_-_-_-_-")
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print(results)


Selected Device: CUDA
_-_-_-_-_-Tokenizing_-_-_-_-_-_-
_-_-_-_-_-Encodings-_-_-_-_-_-


Tokenizing: 100%|██████████| 100/100 [00:07<00:00, 12.75it/s]


_-_-_-_-_-MedicalDataset Creation-_-_-_-_-_-
_-_-_-_-_-Splitting into train and test sets-_-_-_-_-_-


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
`evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead


_-_-_-_-_-Training Started_-_-_-_-_-_-


Epoch,Training Loss,Validation Loss,Mse
1,1007734.900000,1094219.750000,1094219.750000
2,1007534.200000,1093850.750000,1093850.750000
3,1007187.800000,1093295.625000,1093295.500000


{'eval_loss': 1093295.625, 'eval_mse': 1093295.5, 'eval_runtime': 0.2992, 'eval_samples_per_second': 66.835, 'eval_steps_per_second': 10.025, 'epoch': 3.0}


# Clinical BERT

In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
from tqdm import tqdm  # Import tqdm for progress bar

# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("Selected Device: CUDA")
else:
    device = torch.device('cpu')
    print("Selected Device: CPU")

# Load pre-trained ClinicalBERT tokenizer
print("_-_-_-_-_-Tokenizing_-_-_-_-_-_-")
tokenizer = AutoTokenizer.from_pretrained('emilyalsentzer/Bio_ClinicalBERT')

# Example medical text data (X) and continuous target values (y)
texts = X.to_list()
values = y.to_list()

# Tokenize text data with a progress bar
print("_-_-_-_-_-Encodings-_-_-_-_-_-")
encodings = {'input_ids': [], 'attention_mask': []}

# Use tqdm to show progress
for text in tqdm(texts, desc="Tokenizing"):
    encoding = tokenizer(text, truncation=True, padding='max_length', max_length=512)
    encodings['input_ids'].append(encoding['input_ids'])
    encodings['attention_mask'].append(encoding['attention_mask'])

# Convert lists to tensors
encodings = {key: torch.tensor(val) for key, val in encodings.items()}

# Create a dataset class
class MedicalDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, values):
        self.encodings = encodings
        self.values = values

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.values[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.values)

print("_-_-_-_-_-MedicalDataset Creation-_-_-_-_-_-")
dataset = MedicalDataset(encodings, values)

# Split into train and test sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
print("_-_-_-_-_-Splitting into train and test sets-_-_-_-_-_-")
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Load pre-trained ClinicalBERT model with a regression head and move to GPU
model = AutoModelForSequenceClassification.from_pretrained('emilyalsentzer/Bio_ClinicalBERT', num_labels=1).to(device)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',          
    num_train_epochs=3,              
    per_device_train_batch_size=8,   
    per_device_eval_batch_size=8,    
    warmup_steps=500,                
    weight_decay=0.01,               
    logging_dir='./logs',            
    logging_strategy='epoch',
    evaluation_strategy='epoch',
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    save_strategy='epoch'            # Ensure save strategy matches eval strategy
)

# Define custom loss function for regression
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.squeeze()
    mse = ((preds - labels) ** 2).mean().item()
    return {'mse': mse}

# Initialize Trainer
trainer = Trainer(
    model=model,                        
    args=training_args,                 
    train_dataset=train_dataset,         
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Train the model
print("_-_-_-_-_-Training Started_-_-_-_-_-_-")
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print(results)


Selected Device: CUDA
_-_-_-_-_-Tokenizing_-_-_-_-_-_-
_-_-_-_-_-Encodings-_-_-_-_-_-


`resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
Tokenizing: 100%|██████████| 95707/95707 [14:45<00:00, 108.12it/s]


_-_-_-_-_-MedicalDataset Creation-_-_-_-_-_-
_-_-_-_-_-Splitting into train and test sets-_-_-_-_-_-


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

# ClinicalXLNet

In [ ]:
from transformers import XLNetTokenizer, XLNetForSequenceClassification, Trainer, TrainingArguments
import torch
from tqdm import tqdm

# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("Selected Device: CUDA")
else:
    device = torch.device('cpu')
    print("Selected Device: CPU")

# Load pre-trained XLNet tokenizer
print("_-_-_-_-_-Tokenizing_-_-_-_-_-_-")
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')  # Change to a valid model

# Example medical text data (X) and continuous target values (y)
texts = X.to_list()
values = y.to_list()

# Tokenize text data with a progress bar
print("_-_-_-_-_-Encodings-_-_-_-_-_-")
encodings = {'input_ids': [], 'attention_mask': []}

# Use tqdm to show progress
for text in tqdm(texts, desc="Tokenizing"):
    encoding = tokenizer(text, truncation=True, padding='max_length', max_length=512)
    encodings['input_ids'].append(encoding['input_ids'])
    encodings['attention_mask'].append(encoding['attention_mask'])

# Convert lists to tensors
encodings = {key: torch.tensor(val) for key, val in encodings.items()}

# Create a dataset class
class MedicalDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, values):
        self.encodings = encodings
        self.values = values

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.values[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.values)

print("_-_-_-_-_-MedicalDataset Creation-_-_-_-_-_-")
dataset = MedicalDataset(encodings, values)

# Split into train and test sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
print("_-_-_-_-_-Splitting into train and test sets-_-_-_-_-_-")
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Load pre-trained XLNet model with a regression head and move to GPU
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=1).to(device)  # Change to a valid model

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',          
    num_train_epochs=3,              
    per_device_train_batch_size=8,   
    per_device_eval_batch_size=8,    
    warmup_steps=500,                
    weight_decay=0.01,               
    logging_dir='./logs',            
    logging_strategy='epoch',
    evaluation_strategy='epoch',
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    save_strategy='epoch'            
)

# Define custom loss function for regression
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.squeeze()
    mse = ((preds - labels) ** 2).mean().item()
    return {'mse': mse}

# Initialize Trainer
trainer = Trainer(
    model=model,                        
    args=training_args,                 
    train_dataset=train_dataset,         
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Train the model
print("_-_-_-_-_-Training Started_-_-_-_-_-_-")
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print(results)


Selected Device: CUDA
_-_-_-_-_-Tokenizing_-_-_-_-_-_-
_-_-_-_-_-Encodings-_-_-_-_-_-


Tokenizing: 100%|██████████| 100/100 [00:02<00:00, 43.52it/s]


_-_-_-_-_-MedicalDataset Creation-_-_-_-_-_-
_-_-_-_-_-Splitting into train and test sets-_-_-_-_-_-


pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
`evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead


_-_-_-_-_-Training Started_-_-_-_-_-_-


Epoch,Training Loss,Validation Loss,Mse
1,996446.100000,1140349.750000,1140349.750000
2,996105.900000,1139790.750000,1139790.875000
3,995228.900000,1138596.375000,1138596.500000


{'eval_loss': 1138596.375, 'eval_mse': 1138596.5, 'eval_runtime': 0.7821, 'eval_samples_per_second': 25.571, 'eval_steps_per_second': 3.836, 'epoch': 3.0}
